In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '-',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

# Data Generation

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    
    if b > a:
        tmp = a
        a = b
        b = tmp
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [8]:
print(questions[:5], expected[:5])

['92-6   ', '597-12 ', '55-5   ', '429-69 ', '21-0   '] ['86  ', '585 ', '50  ', '360 ', '21  ']


# Processing

In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
split_size = int(TRAINING_SIZE*0.75)
train_x = x[:split_size]
train_y = y[:split_size]
test_x = x[split_size:]
test_y = y[split_size:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(54000, 7, 12)
(54000, 4, 12)
Validation Data:
(6000, 7, 12)
(6000, 4, 12)
Testing Data:
(20000, 7, 12)
(20000, 4, 12)


In [11]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False  True False False
   False]
  [False False False False False False False False False  True False
   False]
  [False False False False False False False  True False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False False  True False False False False False False False False
   False]
  [False False False False False False False  True False False False
   False]]

 [[False False False False  True False False False False False False
   False]
  [False False False False False False False False False  True False
   False]
  [False False False False False False False False False  True False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False  True False False False False
   False]
  [False False False False False False Fal

# Build Model

In [12]:
print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [13]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 61s 1ms/step - loss: 1.7532 - acc: 0.3516 - val_loss: 1.6348 - val_acc: 0.3846
Q 223-48  T 175  ☒ 22  
Q 83-71   T 12   ☑ 12  
Q 538-49  T 489  ☒ 255 
Q 674-76  T 598  ☒ 255 
Q 91-3    T 88   ☒ 11  
Q 68-67   T 1    ☒ 22  
Q 902-403 T 499  ☒ 25  
Q 427-109 T 318  ☒ 22  
Q 509-311 T 198  ☒ 25  
Q 775-78  T 697  ☒ 489 

--------------------------------------------------
Iteration 1
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 47s 869us/step - loss: 1.5533 - acc: 0.4224 - val_loss: 1.4764 - val_acc: 0.4513
Q 346-91  T 255  ☒ 369 
Q 903-0   T 903  ☒ 899 
Q 112-70  T 42   ☒ 11  
Q 323-3   T 320  ☒ 299 
Q 785-48  T 737  ☒ 769 
Q 77-60   T 17

54000/54000 [==============================] - 44s 820us/step - loss: 0.3521 - acc: 0.8894 - val_loss: 0.3226 - val_acc: 0.8985
Q 322-187 T 135  ☑ 135 
Q 73-8    T 65   ☑ 65  
Q 98-36   T 62   ☑ 62  
Q 819-17  T 802  ☑ 802 
Q 442-69  T 373  ☑ 373 
Q 851-5   T 846  ☑ 846 
Q 183-63  T 120  ☒ 129 
Q 909-695 T 214  ☑ 214 
Q 454-52  T 402  ☑ 402 
Q 364-3   T 361  ☑ 361 

--------------------------------------------------
Iteration 14
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 44s 823us/step - loss: 0.2846 - acc: 0.9166 - val_loss: 0.2699 - val_acc: 0.9189
Q 40-11   T 29   ☒ 28  
Q 639-94  T 545  ☑ 545 
Q 343-120 T 223  ☑ 223 
Q 749-73  T 676  ☑ 676 
Q 985-18  T 967  ☒ 958 
Q 669-26  T 643  ☑ 643 
Q 124-77  T 47   ☒ 56  
Q 569-419 T 150  ☒ 160 
Q 612-58  T 554  ☑ 554 
Q 256-8   T 248  ☑ 248 

--------------------------------------------------
Iteration 15
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [===

54000/54000 [==============================] - 44s 822us/step - loss: 0.0626 - acc: 0.9818 - val_loss: 0.0673 - val_acc: 0.9806
Q 64-42   T 22   ☑ 22  
Q 275-46  T 229  ☑ 229 
Q 813-778 T 35   ☑ 35  
Q 963-271 T 692  ☑ 692 
Q 983-798 T 185  ☑ 185 
Q 150-12  T 138  ☑ 138 
Q 651-82  T 569  ☑ 569 
Q 80-72   T 8    ☒ 9   
Q 845-339 T 506  ☑ 506 
Q 856-9   T 847  ☑ 847 

--------------------------------------------------
Iteration 28
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 44s 820us/step - loss: 0.0491 - acc: 0.9874 - val_loss: 0.0639 - val_acc: 0.9795
Q 702-77  T 625  ☑ 625 
Q 991-806 T 185  ☑ 185 
Q 502-53  T 449  ☑ 449 
Q 516-99  T 417  ☑ 417 
Q 620-31  T 589  ☑ 589 
Q 407-164 T 243  ☑ 243 
Q 997-814 T 183  ☑ 183 
Q 926-512 T 414  ☑ 414 
Q 441-19  T 422  ☑ 422 
Q 721-23  T 698  ☑ 698 

--------------------------------------------------
Iteration 29
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [===

54000/54000 [==============================] - 44s 812us/step - loss: 0.0271 - acc: 0.9927 - val_loss: 0.0351 - val_acc: 0.9893
Q 199-37  T 162  ☑ 162 
Q 794-46  T 748  ☑ 748 
Q 260-8   T 252  ☑ 252 
Q 84-8    T 76   ☑ 76  
Q 831-7   T 824  ☑ 824 
Q 732-367 T 365  ☑ 365 
Q 523-316 T 207  ☑ 207 
Q 151-41  T 110  ☑ 110 
Q 881-202 T 679  ☑ 679 
Q 552-472 T 80   ☑ 80  

--------------------------------------------------
Iteration 42
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 43s 805us/step - loss: 0.0262 - acc: 0.9932 - val_loss: 0.1155 - val_acc: 0.9596
Q 776-211 T 565  ☑ 565 
Q 748-646 T 102  ☑ 102 
Q 955-547 T 408  ☑ 408 
Q 896-81  T 815  ☑ 815 
Q 286-60  T 226  ☑ 226 
Q 501-394 T 107  ☑ 107 
Q 805-378 T 427  ☑ 427 
Q 516-95  T 421  ☑ 421 
Q 432-15  T 417  ☑ 417 
Q 306-19  T 287  ☑ 287 

--------------------------------------------------
Iteration 43
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [===

54000/54000 [==============================] - 43s 802us/step - loss: 0.0115 - acc: 0.9979 - val_loss: 0.0193 - val_acc: 0.9943
Q 828-68  T 760  ☑ 760 
Q 168-24  T 144  ☑ 144 
Q 49-27   T 22   ☑ 22  
Q 907-5   T 902  ☑ 902 
Q 807-676 T 131  ☑ 131 
Q 808-50  T 758  ☑ 758 
Q 887-1   T 886  ☑ 886 
Q 690-14  T 676  ☑ 676 
Q 725-20  T 705  ☑ 705 
Q 915-7   T 908  ☑ 908 

--------------------------------------------------
Iteration 56
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 43s 800us/step - loss: 0.0101 - acc: 0.9981 - val_loss: 0.0240 - val_acc: 0.9921
Q 512-116 T 396  ☑ 396 
Q 944-47  T 897  ☑ 897 
Q 542-455 T 87   ☒ 77  
Q 123-52  T 71   ☑ 71  
Q 566-539 T 27   ☑ 27  
Q 177-16  T 161  ☑ 161 
Q 838-561 T 277  ☑ 277 
Q 237-0   T 237  ☑ 237 
Q 315-28  T 287  ☑ 287 
Q 874-64  T 810  ☑ 810 

--------------------------------------------------
Iteration 57
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [===

54000/54000 [==============================] - 44s 807us/step - loss: 0.0061 - acc: 0.9992 - val_loss: 0.0167 - val_acc: 0.9953
Q 325-230 T 95   ☑ 95  
Q 873-45  T 828  ☑ 828 
Q 191-8   T 183  ☑ 183 
Q 580-561 T 19   ☒ 29  
Q 897-842 T 55   ☑ 55  
Q 926-254 T 672  ☑ 672 
Q 664-20  T 644  ☑ 644 
Q 852-4   T 848  ☑ 848 
Q 295-98  T 197  ☑ 197 
Q 426-313 T 113  ☑ 113 

--------------------------------------------------
Iteration 70
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 44s 808us/step - loss: 0.0238 - acc: 0.9928 - val_loss: 0.0267 - val_acc: 0.9914
Q 684-105 T 579  ☑ 579 
Q 852-56  T 796  ☑ 796 
Q 346-1   T 345  ☑ 345 
Q 552-220 T 332  ☑ 332 
Q 920-199 T 721  ☑ 721 
Q 787-68  T 719  ☑ 719 
Q 170-87  T 83   ☑ 83  
Q 955-777 T 178  ☑ 178 
Q 775-267 T 508  ☑ 508 
Q 253-61  T 192  ☑ 192 

--------------------------------------------------
Iteration 71
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [===

54000/54000 [==============================] - 44s 814us/step - loss: 0.0047 - acc: 0.9994 - val_loss: 0.0172 - val_acc: 0.9940
Q 448-344 T 104  ☑ 104 
Q 697-542 T 155  ☑ 155 
Q 776-691 T 85   ☑ 85  
Q 533-24  T 509  ☑ 509 
Q 351-153 T 198  ☑ 198 
Q 713-181 T 532  ☑ 532 
Q 615-43  T 572  ☑ 572 
Q 993-210 T 783  ☑ 783 
Q 919-311 T 608  ☑ 608 
Q 750-361 T 389  ☑ 389 

--------------------------------------------------
Iteration 84
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 44s 808us/step - loss: 0.0278 - acc: 0.9918 - val_loss: 0.0174 - val_acc: 0.9948
Q 392-79  T 313  ☑ 313 
Q 194-4   T 190  ☒ 180 
Q 219-127 T 92   ☒ 91  
Q 683-95  T 588  ☑ 588 
Q 924-56  T 868  ☑ 868 
Q 372-79  T 293  ☑ 293 
Q 258-82  T 176  ☑ 176 
Q 98-36   T 62   ☑ 62  
Q 946-920 T 26   ☒ 16  
Q 997-94  T 903  ☑ 903 

--------------------------------------------------
Iteration 85
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [===

54000/54000 [==============================] - 46s 844us/step - loss: 0.0028 - acc: 0.9999 - val_loss: 0.0116 - val_acc: 0.9964
Q 983-660 T 323  ☑ 323 
Q 818-60  T 758  ☑ 758 
Q 363-283 T 80   ☒ 70  
Q 155-1   T 154  ☑ 154 
Q 686-88  T 598  ☑ 598 
Q 403-32  T 371  ☑ 371 
Q 392-0   T 392  ☑ 392 
Q 684-66  T 618  ☑ 618 
Q 904-400 T 504  ☑ 504 
Q 186-2   T 184  ☑ 184 

--------------------------------------------------
Iteration 98
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 37s 690us/step - loss: 0.0025 - acc: 0.9999 - val_loss: 0.0129 - val_acc: 0.9962
Q 180-7   T 173  ☑ 173 
Q 642-43  T 599  ☑ 599 
Q 78-32   T 46   ☑ 46  
Q 564-9   T 555  ☑ 555 
Q 497-35  T 462  ☑ 462 
Q 332-15  T 317  ☑ 317 
Q 911-843 T 68   ☑ 68  
Q 947-85  T 862  ☑ 862 
Q 823-1   T 822  ☑ 822 
Q 679-1   T 678  ☑ 678 

--------------------------------------------------
Iteration 99
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [===

# Testing

In [14]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 
#####################################################

predictions = model.predict_classes(test_x)
print(predictions)


    

MSG : Prediction
[[ 5  4  4  0]
 [ 7 10  4  0]
 [11  7  3  0]
 ...
 [ 9  2 11  0]
 [ 5  7  7  0]
 [ 8 11  0  0]]


In [15]:
predict_right = 0
for i in range(test_y.shape[0]):
    predict_y = ctable.decode(predictions[i],calc_argmax=False)
    correct_y = ctable.decode(test_y[i])
    if predict_y == correct_y:
        predict_right+=1
accuracy = predict_right / test_y.shape[0]
print("subtraction model accuracy:",accuracy)
    

subtraction model accuracy: 0.9884
